[View in Colaboratory](https://colab.research.google.com/github/Santosh-Gupta/BookCrossingsRecommender/blob/master/BookCrossingsTraining.ipynb)

Now that we have the refined data and book info files produced in the BookCrossingsDataAnalysis.ipynb notebook, we can train models on the data. 

In [0]:
import re
from collections import Counter
import time

import numpy as np

from __future__ import print_function
import collections
import math
import numpy as np
import random
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

import os

from urllib.request import urlretrieve
from os.path import isfile, isdir
import zipfile
import tensorflow as tf
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
import pandas as pd

!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import os
import sys
from numpy import genfromtxt

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

vocabulary_size =  40988 #number of books

tf.logging.set_verbosity(tf.logging.ERROR)

Input Google drive file ID for the refined ratings file produced in BookCrossingsDataAnalysis.ipynb notebook

In [5]:
dl_id = input("Enter Gdrive file ID for Refined Ratings: ") #Book Crossing 1mtg_TdC4FBlbQh30Yt1-Rxcyu2Q38Axv


# DOWNLOAD ZIP
print ("Downloading  file")
myDownload = drive.CreateFile({'id': dl_id})
myDownload.GetContentFile('DlRefinedRatings.csv')
print( os.listdir() )

my_data = genfromtxt('DlRefinedRatings.csv', delimiter=',' ,  dtype=int, skip_header=1)

print(my_data[0:15])


Enter Gdrive file ID for Refined Ratings: 1mtg_TdC4FBlbQh30Yt1-Rxcyu2Q38Axv
['datalab', '.config', '.local', '.ipython', 'DlRefinedRatings.csv', '.keras', '.forever', '.cache']
[[1082427  259004     784       8      22      54      17]
 [1082450  259004   24422       7      22      91      -1]
 [1082415  259004    3085       7      22     183      -1]
 [1082363  259004   24458       6      22     250      -1]
 [1082357  259004    2465       6      22     251      -1]
 [1082426  259004    6411       8      22     258      -1]
 [1082425  259004   14199       8      22     260      -1]
 [1082422  259004    5035      10      22     264      -1]
 [1082391  259004    3447       9      22     267      -1]
 [1082412  259004    2483       9      22     281      -1]
 [1082451  259004   24457       8      22     304      -1]
 [1082440  259004   13666       6      22     352      -1]
 [1082378  259004    3217       7      22     414      -1]
 [1082361  259004    5370       7      22     419      -

Input Google drive file ID for the book info file produced in BookCrossingsDataAnalysis.ipynb notebook

In [6]:
dl_id = input("Enter Gdrive file ID for Book info: ") #Book Crossing  1thtwqN1fhyiLeEnmj51ZhEaW5DgonYgC


# DOWNLOAD ZIP
print ("Downloading  file")
myDownload = drive.CreateFile({'id': dl_id})
myDownload.GetContentFile('BookInfo.csv')
print( os.listdir() )

Enter Gdrive file ID for Book info: 1thtwqN1fhyiLeEnmj51ZhEaW5DgonYgC
['datalab', '.config', '.local', '.ipython', 'BookInfo.csv', 'DlRefinedRatings.csv', '.keras', '.forever', '.cache']


Create dictionaries to look up Authors and Book Titles from the EmbedID

In [7]:
b = pd.read_csv( 'BookInfo.csv' )
b.head(30)
bookDictionary = b.set_index('EmbedID').to_dict()['Book_Title']
bookDictionary[7]
authorDictionary = b.set_index('EmbedID').to_dict()['Author']
authorDictionary[7]

'Dan Brown'

This code section produces a batch of input data and corresponding labels. 

In [0]:
data_index = 0
epoch_index = 0
recEpoch_indexA = 0 #Used to help keep store of the total number of epoches with the models

def generate_batch(batch_size, inputCount): 
  
    global data_index, epoch_index
    
    span = inputCount 
    batch = np.ndarray(shape=(batch_size, span), dtype=np.int32) 
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
    
    userbracketDF = my_data[data_index: data_index+ my_data[data_index, 6], 2 ]
    headernumber = my_data[:,6]
    assert headernumber[data_index] != -1, 'data_index in incorrect spot. Most likely fix is to reset data_index to 0'
        
    n=0
    while n < batch_size:
      b=0
      while b < len(userbracketDF) and n < batch_size :
        #print('n is', n)
        if b== 0:
          potentialBatch=userbracketDF[b+1:len(userbracketDF)]
        else:
          firsthalf = userbracketDF[0:b]
          secondhalf = userbracketDF[b+1:len(userbracketDF)]
          potentialBatch = np.concatenate((firsthalf, secondhalf), axis=0)
        potentialBatchSet = set(potentialBatch)
        if inputCount > len(potentialBatchSet):
          print('potentialBatchSet is ', potentialBatchSet)
          print('data_index is ', data_index)
          raise Exception('Dataset is for a smaller Cbow Window, data needs to be checked') 
        batch[n] = random.sample(potentialBatchSet,  inputCount)
        labels[n,0]=userbracketDF[b]
        n=n+1
        b=b+1
      
      data_index = (data_index + headernumber[data_index] ) % len(headernumber)
      
      if data_index == 0:
        epoch_index = epoch_index + 1
        print('Completed %d Epochs' % epoch_index)
      
      #print('data_index is ', data_index)
      assert headernumber[data_index] != -1, 'data_index in incorrect spot'
      userbracketDF = my_data[data_index: data_index+ my_data[data_index, 6], 2 ]

    return batch, labels     
      
# here, goes = generate_batch(20, 1) # to do next, insert %len(headernumber)
# print('batch', here)
# print('labels', goes)

Define the graph

In [0]:
batch_size = 128 #2^8

embedding_size = 200 # 2^8 Dimension of the embedding vector.
num_inputs =1
#skip_window = 1 # How many words to consider left and right.
#num_skips = 2 # How many times to reuse an input to generate a label.
                      
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 60 # Random set of words to evaluate similarity on.
valid_window = 5000 # Only pick dev samples in the head of the distribution.
#valid_examples = np.array(random.sample(range(valid_window), valid_size))
# Random set of words to evaluate similarity on.
num_sampled = 65 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default(): #took out " , tf.device('/cpu:0')"
 
  valid_examples = np.array(random.sample(range(1, valid_window), valid_size)) #put inside graph to get new words each time
  #for same words each time keep outside graph
  
  # Input data.
  train_dataset = tf.placeholder(tf.int32, shape=[batch_size, num_inputs ])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  valid_datasetSM = tf.constant(valid_examples, dtype=tf.int32)

  epochCount = tf.get_variable( 'epochCount', initializer= 0) #to store epoch count to total # of epochs are known
  update_epoch = tf.assign(epochCount, epochCount + 1)
  # Variables.
  #embeddings = tf.Variable(
  #  tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  embeddings = tf.get_variable( 'embeddings', 
    initializer= tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  #softmax_weights = tf.Variable(
  #  tf.truncated_normal([vocabulary_size, embedding_size],
  #                       stddev=1.0 / math.sqrt(embedding_size)))
  softmax_weights = tf.get_variable( 'softmax_weights',
    initializer= tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
  
  softmax_biases = tf.get_variable('softmax_biases', 
    initializer= tf.zeros([vocabulary_size]),  trainable=False )
  #Made softmax biases untrainable because they do not seem to give significant
  #improvement. Also allows for a 2nd set of embeddings to analyze. 
  #https://stats.stackexchange.com/questions/249565/word2vec-neural-network-bias-units/250464
  #Both Richard Sorcher and Andrew Ng's NLP lectures do not contain a bias term
  #in their equations. 
  
  # Model.
  # Look up embeddings for inputs.
  embed = tf.nn.embedding_lookup(embeddings, train_dataset) #train data set is
  #the center word for skip-gram mode. So we're looking up the vector for that
  #center word. Gets the embeddings for every word in the batch
  # Compute the softmax loss, using a sample of the negative labels each time.
  embed_reshaped = tf.reshape( embed, [batch_size*num_inputs, embedding_size] )
  
  
  segments= np.arange(batch_size).repeat(num_inputs)
  #segments= np.arange(embedding_size).repeat(num_inputs)
  #should be batch size, not embedding size
  
  averaged_embeds = tf.segment_mean(embed_reshaped, segments, name=None)
  
  #def loss_function():
    #return tf.reduce_mean( tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=averaged_embeds,
                               #labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))
  
  loss = tf.reduce_mean(
    tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=averaged_embeds,
                               labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))
            #fun fact remove_accidental_hits: A bool. whether to remove "accidental hits" where 
            #a sampled class equals one of the target classes. Default is True.

  # Optimizer.
  # Note: The optimizer will optimize the softmax_weights AND the embeddings.
  # This is because the embeddings are defined as a variable quantity and the
  # optimizer's `minimize` method will by default modify all variable quantities 
  # that contribute to the tensor it is passed.
  # See docs on `tf.train.Optimizer.minimize()` for more details.
  optimizer = tf.train.AdagradOptimizer(0.03).minimize(loss) #Original learning rate was 1.0
  
  # Compute the similarity between minibatch examples and all embeddings.
  # We use the cosine distance:
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
  normSM = tf.sqrt(tf.reduce_sum(tf.square(softmax_weights), 1, keepdims=True))
  
  normalized_embeddings = embeddings / norm
  normalized_embeddingsSM = softmax_weights / normSM
  
  valid_embeddings = tf.nn.embedding_lookup(
    normalized_embeddings, valid_dataset) #A bunch of normalized embeddings, randomly selected when valid_examples was initilized. 
  valid_embeddingsSM = tf.nn.embedding_lookup(
    normalized_embeddingsSM, valid_datasetSM)
  
  similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings)) #multiplies each normalized embeds from valid_examples with all 
                                                                                                                        #other normalized embeds. Used to find most similiar embeds later
  similaritySM = tf.matmul(valid_embeddingsSM, tf.transpose(normalized_embeddingsSM)) 
  #important to note that to note that throughout all these matrix operations, indices of the original embedding and SM matrices are kept in tact
  #this is important for when we want to match an embedding with it's corresponding book, since the book dictionary uses the index as the key
                                                                                                                              
  saver = tf.train.Saver()

This zip folder is used to save the checkpoints and zip them into a zipfile so that they can be easily uploaded to Google Drive

In [0]:
def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

If you would like to continue from a previous Tensorflow checkpoint, select 'y' and put the Google file ID for the checkpoint

In [11]:
loadModel = input("Would you like to load a checkpoint? Type y or n: ") 

if loadModel == 'y':
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  
  zip_id = input("Enter Gdrive file ID for tensorflow models: ") 

  if not os.path.exists('checkpointsBook2VecCbowWindow1Downloaded'):
      os.makedirs('checkpointsBook2VecCbowWindow1Downloaded')

  # DOWNLOAD ZIP
  print ("Downloading zip file")
  myzip = drive.CreateFile({'id': zip_id})
  myzip.GetContentFile('model.zip')

  # UNZIP ZIP
  print ("Uncompressing zip file")
  zip_ref = zipfile.ZipFile('model.zip', 'r')
  zip_ref.extractall('checkpointsBook2VecCbowWindow1Downloaded/')
  zip_ref.close()

  print( os.getcwd() )
  print( os.listdir('./checkpointsBook2VecCbowWindow1Downloaded') )
  
  #with tf.Session(graph=graph) as session:
    #saver = tf.train.import_meta_graph('./checkpointsBook2VecCbowWindow3Downloaded/bookVec.ckpt.meta')
    #saver.restore(session, './checkpointsBook2VecCbowWindow3Downloaded/bookVec.ckpt' )

Would you like to load a checkpoint? Type y or n: y
Enter Gdrive file ID for tensorflow models: 1DjPW4rpMP5Dl_7IQgTI_5NkzH2ResTeD
Uncompressing zip file
/content
['bookVec.ckpt.data-00000-of-00001', 'checkpoint', 'bookVec.ckpt.index', 'bookVec.ckpt.meta']


Run the Tensorflow graph in a session.

In [0]:
num_steps = 1000000

if 'loadModel' not in locals() and 'loadModel' not in globals():
  loadModel = 'n'

uploadModel = drive.CreateFile() #used to upload checkpoints when graph is run

with tf.Session(graph=graph) as session:
  
  if loadModel == 'y':
    saver.restore(session, './checkpointsBook2VecCbowWindow1Downloaded/bookVec.ckpt' )
  else:  
    tf.global_variables_initializer().run() #Don't initalize variables after a checkpoint has been restored
  
  print('Initialized')
  average_loss = 0
  saveIteration = 1
  for step in range(1, num_steps):
    
    batch_data, batch_labels = generate_batch(
      batch_size, num_inputs)
    #print(batch_data)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    #graph_batch = session.run([train_dataset], feed_dict=feed_dict) 
    #print(graph_batch)
    _, l = session.run([optimizer, loss], feed_dict=feed_dict) 

    average_loss += l
    if step % 50000 == 0:
      if step > 0:
        average_loss = average_loss / 50000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    
    if step % 100000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = bookDictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1] #Returns index of most similiar values. Starts from 1 because index 0 will just be the same book
        log = 'Nearest to %s : | ' % valid_word
        for k in range(top_k):
          close_word = bookDictionary[nearest[k]]
          log = '%s %s | ' % (log, close_word) #Prints out most similiar books, in descending order from most to least similiar
        print(log)
        if gauth.credentials is None:
        # Authenticate if they're not there
          gauth.LocalWebserverAuth()
        elif gauth.access_token_expired:
          # Refresh them if expired
          print( "Google Drive Token Expired, Refreshing")
          gauth.Refresh()
        else:
          # Initialize the saved creds
          gauth.Authorize()
      
    if step % 150000 == 0:
      recEpoch_indexA =  epoch_index - recEpoch_indexA #how much did the epoch_index since it was last checked
      #epochCount = tf.add(  epochCount, recEpoch_indexA, name=None )
      for nE in range(0, recEpoch_indexA ):
        session.run(update_epoch) #session run calls tend to be huge bottlenecks, keep in mind while determining the frequency
      recEpoch_indexA = epoch_index
      print('recEpoch_indexA is', recEpoch_indexA)
      print( 'epochCount.eval() is ', epochCount.eval() )
      print('epoch_index is ' , epoch_index)
      
      save_path = saver.save(session, "B2VBookCrossingsckpt/bookVec.ckpt") #Save checkpoint
      
      auth.authenticate_user()
      gauth = GoogleAuth() #Gdrive authenticion code placed here since it expires after some time
      gauth.credentials = GoogleCredentials.get_application_default()
      drive = GoogleDrive(gauth) 
      uploadModel = drive.CreateFile() #Need to also create drive object with updated authenticion
      
      chptName = 'B2VBookCrossingsckpt'+str(saveIteration)
      zipfolder(chptName, 'B2VBookCrossingsckpt') #2nd input is the folder name of the save path
      uploadModel.SetContentFile(chptName+".zip")
      uploadModel.Upload()
      
      print("Checkpoint uploaded to Google Drive")
      saveIteration += 1
      os.remove(chptName+".zip") #Remove checkpoint zip file after upload

                
  #extract the embeddings
  final_embeddings = normalized_embeddings.eval() #extract the embeddings
  final_embeddingsSM = normalized_embeddingsSM.eval()

Initialized
Completed 1 Epochs
Completed 2 Epochs
Completed 4 Epochs
Completed 5 Epochs
Completed 6 Epochs
Completed 7 Epochs
Completed 8 Epochs
Completed 9 Epochs
Completed 10 Epochs
Completed 11 Epochs
Completed 12 Epochs
Completed 13 Epochs
Completed 14 Epochs
Completed 15 Epochs
Completed 16 Epochs
Completed 17 Epochs
Completed 18 Epochs
Completed 19 Epochs
Completed 20 Epochs
Completed 21 Epochs
Completed 22 Epochs
Completed 23 Epochs
Completed 24 Epochs
Completed 25 Epochs


The trained embeddings are analyzed in much more detail in the BookCrossingsAnalyzeEmbeddings.ipynb file. 